# Comparison between numerical stability curves

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image

import numpy as np
from numpy.polynomial import polynomial as pol
import pickle

### Load numerical stability curves

In [2]:
methods = ['Cholesky', 'CGLS', 'SOB17', 'TOB20', 
           'deconv_unstable', 'deconv_suboptimal', 'deconv_optimal', 'deconv_overshoot']

In [3]:
stability_curves = []

In [4]:
for method in methods:
    with open(method+'_stability.pickle', 'rb') as f:
        stability_curves.append(pickle.load(f))

### Estimate the slopes of numerical stability curves

In [5]:
lines = []

# define the number of points forming each curve
N = len(stability_curves[0]['model'])

for curve in stability_curves:
    # define the matrix for linear regression
    A = pol.polyvander(curve['model'], 1)
    # define the normal equations
    ATA = A.T@A
    ATd = A.T@curve['data']
    # estimate the coefficients of the straight line
    lines.append(np.linalg.solve(ATA, ATd))

In [16]:
for (method, line) in zip(methods, lines):
    print('kappa_{}: {}'.format(method, line[1]))

kappa_Cholesky: 2.2947744533320655
kappa_CGLS: 2.3803420618531055
kappa_SOB17: 3.2502345696885917
kappa_TOB20: 2.37994276417169
kappa_deconv_unstable: 4.827534605322033
kappa_deconv_suboptimal: 1.5929540338731782
kappa_deconv_optimal: 1.1173115188557763
kappa_deconv_overshoot: 0.9338865590992813


### Plot the results

In [ ]:
names = ['$\mathtt{Cholesky}$',
         '$\mathtt{CGLS}$',
         '$\mathtt{SOB17}$',
         '$\mathtt{TOB20}$',
         '$\mathtt{deconv}$ (unstable)',
         '$\mathtt{deconv}$ (suboptimal)',
         '$\mathtt{deconv}$ (optimal)',
         '$\mathtt{deconv}$ (overshoot)']

colors = ['black',
          'red',
          'blue',
          'purple',
          'lightgrey', 
          'darkgrey', 
          'magenta',
          'dimgrey']

linestyles = ['-', '-', '-', '-', ':', ':', '-', ':']

markers = ['o', 
           'x',
           'v',
           '^',
           'D',
           'D',
           'D',
           'D']

In [ ]:
Dp_min = 0
Dp_max = 0.7

fig, ax = plt.subplots(figsize=(8,5))

for (color, linestyle, marker, method, name, line) in zip(colors, linestyles, markers, methods, names, lines):
    ### Open the stability curve
    file_name = method+'_stability.pickle'
    with open(file_name, 'rb') as f:
        numerical_stability_curve = pickle.load(f)
    ### Plot the stability curve line
    Dd_min = line[0] + line[1]*Dp_min
    Dd_max = line[0] + line[1]*Dp_max
    ax.plot([
        Dp_min, Dp_max], [Dd_min, Dd_max],
        linestyle=linestyle, color=color, marker='')
    ### Plot the stability curve points
    ax.plot(
        numerical_stability_curve['model'], 
        numerical_stability_curve['data'], 
        linestyle='', color=color, marker=marker, alpha=1, markersize=4, label=name)

ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)
ax.set_xlabel('$\Delta p^{\ell}$', fontsize=18)
ax.set_ylabel('$\Delta d^{\ell}$', fontsize=18)
ax.set_xlim(Dp_min, Dp_max)
ax.grid(True)
ax.legend(loc='best', fontsize=10)
plt.tight_layout()
plt.savefig('../manuscript/Fig/stability-comparison.png', dpi= 300)
plt.show()